<img src='sharif_logo.png' alt="SUT logo" width=150 height=150 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
 Deep Learning <br>
<font color=2565AE size=5>
Computer Engineering Department - Spring 2025  <br>
<font color=3C99D size=5>
          Homework 2:  <br>
<font color=696880 size=4>
           
    

**Name**: Nikan Vasei  
  
**Student Code**: 400105303

<font color='cyan'>

## Some Notes & Reflections

Throughout this project, I implemented and evaluated several reasoning strategies as intended, including Chain of Thought (CoT) prompting, best-of-n, beam search, and Self-Refinement, to improve the model's performance on mathematical problems. While the overall structure and logic of the implementation were solid, a number of practical limitations affected the quality of the results:

- **Frequent Colab Disconnects**: Repeated disconnections forced me to restart the notebook multiple times, which interrupted long-running processes and slowed down experimentation. It took so long for me to be able to fully run the notebook :(
    - Also hosting the model via VLLM occasionally caused disconnections, further complicating the evaluation loop.

- **Token Limitations**: Due to output length restrictions (which was about 5k), some model completions were cut off prematurely, often missing the final answer in the required \\boxed{} format. This also causes the benchmark to count that answer as a wrong one (Those kinds of answers are shown as `None` in cell outputs).

- **Limited Parameter Tuning**: Parameters such as temperature, top_p, and beam width had a significant impact on output quality, but time and platform constraints limited my ability to fine-tune them thoroughly. (Obviously we could get better results, if we had fine-tuned them more.)

Despite these challenges, the implementation demonstrates how each reasoning strategy influences model performance. The results show promising trends and highlight the strengths and trade-offs of each approach, especially in terms of coherence, completeness, and correctness of answers.

# Assignment Overview

In this assignment, you will explore inference scaling techniques in large language models (LLMs) and evaluate their performance using the Math Benchmark. Throughout the notebook, you will learn about several inference methods, including:

- **Chain-of-Thought (CoT):** A method where the model generates intermediate reasoning steps before providing the final answer.
- **Best-of-n Sampling:** An approach that generates multiple candidate responses and selects the best one based on a scoring function.
- **Beam Search:** A technique that expands several possible sequences simultaneously, choosing the most promising ones based on probability.
- **Self-Refinement:** An iterative process where the model revises its output to improve accuracy and coherence.

The **Math Benchmark** is a suite of challenging mathematical problems designed to test the reasoning and problem-solving capabilities of LLMs. The benchmark includes a variety of questions ranging from basic arithmetic and algebra to more advanced topics such as geometry and calculus. For example, you might be asked to solve an equation like `2x + 5 = 15` or compute the derivative of a function, tasks that assess the model's ability to handle both straightforward and complex mathematical queries.

By the end of this assignment, you will have:
- Gained a deeper understanding of inference time scaling methods in LLMs.
- Compared the effectiveness of different inference techniques using a rigorous math evaluation framework.

Let's dive into the notebook and begin exploring how these methods perform on a challenging set of math problems!


## vLLM: Accelerated Inference Engine for LLMs

vLLM is an open-source project designed to optimize the loading and inference of large language models. By leveraging advanced memory management techniques and dynamic batching, vLLM significantly speeds up the inference process, making it easier to deploy and experiment with LLMs even on hardware with limited resources
So we use vLLM to get results faster.

# installing Dependencies

In [1]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

In [2]:
!pip install transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
gcsfs 2025.3.2 

In [ ]:
!pip install --upgrade numpy
import os
os.kill(os.getpid(), 9)


This command launches a vLLM inference server with:
- Model: `DeepSeek-R1-Distill-Qwen-1.5B`
- Port: `8000` (default API endpoint)
- Precision: `half` (FP16) for memory efficiency
- Max context length: `3192` tokens

**Note:**  
🔹 Ensure you're using a GPU runtime (T4 or better) in Colab  
🔹 Only run the next cell if this one executes successfully


In [ ]:
!vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"   --port 8000   --dtype=half   --max-model-len 3192

* this cell lunches model in background using vllm

In [3]:
!nohup vllm serve "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" --port 8000 --dtype=half --max-model-len 5192 &

nohup: appending output to 'nohup.out'


## LLM Query Function

* This Python function sends prompts to a locally-hosted LLM API and returns the generated response
* you can change max_tokens and temperature as you want



In [4]:
import requests
def get_llm_response(prompt):
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 2500,
    "temperature": 0.6
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Test response generation
- testing model with some Math benchmark quesions

In [ ]:
# TODO: Generate a response with these Math benchmark quesions
question1 = "How many positive whole-number divisors does 196 have?"
# real answer : 9
question2 = "What is the distance, in units, between the points $(2, -6)$ and $(-4, 3)$? Express your answer in simplest radical form."
# real answer = 3\\sqrt{13}
question3 = "Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$"
# real answer = p - q

prompt = f"""Please answer each of the following math questions step-by-step and briefly.

- {question1}

- {question2}

- {question3}

Also make sure to divide each answer by a dash so that they would be easy to follow.

"""

response = get_llm_response(prompt)
print(response)

Okay, so I have these three math questions to solve. Let me take them one by one and think through each step carefully.

**First question: How many positive whole-number divisors does 196 have?**

Alright, divisors. Hmm, I remember that to find the number of divisors of a number, I should start by factoring it into its prime factors. Then, using the exponents in the prime factorization, I can apply the formula for the number of divisors.

So, let's factor 196. I know that 196 is a perfect square because 14 squared is 196. Let me confirm that: 14 times 14 is 196. So, 196 = 14^2. But 14 itself is 2 times 7, so 196 can be written as (2*7)^2, which is 2^2 * 7^2.

Wait, hold on. Is that right? Let me check: 2^2 is 4, 7^2 is 49, and 4*49 is 196. Yes, that's correct.

So the prime factorization of 196 is 2^2 * 7^2. Now, the formula for the number of divisors is to take the exponents, add one to each, and then multiply them together. So for 2^2, the exponent is 2, so 2+1=3. Similarly, for 7^2,

# Math Benchmark Evaluation

This cell is dedicated to evaluating the performance of inference scaling methods on the Math Benchmark dataset. The process works as follows:

- **Dataset Loading:** It loads the MATH-500 dataset, which contains a set of challenging math problems along with their correct solutions.
- **Answer Extraction:** The `extract_answer` function is used to parse and extract the final answer from the generated responses. This function specifically looks for a LaTeX-style format (using `\boxed{...}`) to reliably pinpoint the answer.
- **Normalization and Comparison:** Before comparing, both the predicted answer and the ground truth are normalized using several functions. These functions handle different mathematical expressions, such as fractions, matrices, and algebraic expressions, ensuring that the comparison is fair and accurate regardless of formatting differences.
- **Evaluation Loop:** For each problem:
  - The ground truth answer is extracted from the provided solution.
  - A response is generated by the LLM using a designated function.
  - The predicted answer is then extracted and compared against the ground truth.
  - The results for each problem, including whether the predicted answer is correct, are saved for later analysis.
- **Results Analysis:** After processing all problems, the cell aggregates the results and prints a summary, including the total number of problems evaluated, the number of correct answers, and the overall accuracy.

This evaluation method ensures that the output of each inference technique (such as Chain-of-Thought, Best-of-n, Beam Search, and Self-Refinement) is consistently measured against the Math Benchmark, without altering the original answers or evaluation logic.

**Note:**  

🔹 you don't need to modify this cell. Only rewrite the evaluation function portion then

🔹 you need to run this cell before evaluating.


In [5]:
import json
import os
import re
from typing import Dict, Optional, Union
from datasets import load_dataset
from tqdm import tqdm
import torch

# Load the MATH-500 dataset
def load_math500_dataset():
    dataset = load_dataset("HuggingFaceH4/MATH-500")["test"]
    return dataset

# Extract the last boxed answer from text
def extract_answer(response: str) -> Optional[str]:
    if not response:
        return None
    start_idx = response.rfind('\\boxed{')
    if start_idx == -1:
        return None
    brace_count = 1
    pos = start_idx + 7  # length of '\boxed{'
    while pos < len(response) and brace_count > 0:
        if response[pos] == '{':
            brace_count += 1
        elif response[pos] == '}':
            brace_count -= 1
        pos += 1
    if brace_count == 0:
        answer = response[start_idx + 7:pos - 1]
        return answer.strip()
    return None

# Normalization and comparison functions (unchanged from original)
def normalize_number(num_str: str) -> str:
    try:
        cleaned = re.sub(r'[,\$\\]|\s*(?:cm|m|kg|ft|in|lb|oz|ml|L)$|\s*\\text{[^}]+}', '', num_str).strip()
        if cleaned.startswith('.'):
            cleaned = '0' + cleaned
        num = float(cleaned)
        if abs(num) < 1 and '.' in cleaned:
            decimal_places = len(cleaned.split('.')[1])
            format_str = f"{{:.{decimal_places}f}}"
            result = format_str.format(num)
        else:
            result = str(num)
        return result
    except:
        return num_str

def numerically_equal(str1: str, str2: str) -> bool:
    try:
        return abs(float(str1) - float(str2)) < 1e-10
    except:
        return False

def normalize_fraction(fraction_str: str) -> str:
    try:
        fraction_str = fraction_str.replace('\\dfrac', '\\frac')
        fraction_str = ''.join(fraction_str.split())
        fraction_str = re.sub(r'\s*\\text{[^}]+}', '', fraction_str)
        mixed_brace = re.match(r'^\\frac(\d+)\{(\d+)\}$', fraction_str)
        if mixed_brace:
            num, den = mixed_brace.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        no_braces = re.match(r'^\\frac(\d+)(\d+)$', fraction_str)
        if no_braces:
            num, den = no_braces.groups()
            return f"\\frac{{{num}}}{{{den}}}"
        if '/' in fraction_str and not any(c in fraction_str for c in '\\{}'):
            num, den = fraction_str.split('/')
            return f"\\frac{{{num.strip()}}}{{{den.strip()}}}"
        standard = re.match(r'^\\frac\{([^{}]+)\}\{([^{}]+)\}$', fraction_str)
        if standard:
            num, den = standard.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return fraction_str

def normalize_matrix_entry(entry: str) -> str:
    entry = ''.join(entry.split())
    if '/' in entry and not any(c in entry for c in '\\{}'):
        if entry.startswith('-'):
            num, den = entry[1:].split('/')
            return f"-{num.strip()}/{den.strip()}"
        else:
            num, den = entry.split('/')
            return f"{num.strip()}/{den.strip()}"
    entry = entry.replace('\\dfrac', '\\frac')
    frac_match = re.match(r'^(-)?\\frac\{(\d+)\}\{(\d+)\}$', entry)
    if frac_match:
        sign, num, den = frac_match.groups()
        sign = sign if sign else ''
        return f"{sign}{num}/{den}"
    return entry

def normalize_matrix(matrix_str: str) -> str:
    try:
        matrix_str = ''.join(matrix_str.split())
        match = re.match(r'^\\begin\{pmatrix\}(.*?)\\end\{pmatrix\}$', matrix_str)
        if not match:
            return matrix_str
        content = match.group(1)
        rows = content.split('\\\\')
        normalized_rows = []
        for row in rows:
            if '&' in row:
                entries = [normalize_matrix_entry(entry) for entry in row.split('&')]
            else:
                entries = [normalize_matrix_entry(row)]
            normalized_rows.append('&'.join(entries))
        result = "\\begin{pmatrix}" + "\\\\".join(normalized_rows) + "\\end{pmatrix}"
        return result
    except:
        return matrix_str

def normalize_algebraic_expression(expr: str) -> str:
    try:
        expr = ''.join(expr.split())
        monomial_match = re.match(r'^(-?\d*\.?\d*)?([a-zA-Z])(?:\^(-?\d+))?$', expr)
        if monomial_match:
            coeff, var, exp = monomial_match.groups()
            coeff = coeff if coeff and coeff not in ['+', '-'] else ('1' if not coeff else '-1')
            exp = exp if exp else '1'
            if coeff == '1' and exp == '1':
                return var
            elif coeff == '1':
                return f"{var}^{exp}"
            elif coeff == '-1' and exp == '1':
                return f"-{var}"
            elif coeff == '-1':
                return f"-{var}^{exp}"
            elif exp == '1':
                return f"{coeff}{var}"
            else:
                return f"{coeff}{var}^{exp}"
        pi_term_match = re.match(r'^(-?\d*\.?\d*)\\?pi$', expr)
        if pi_term_match:
            coeff = pi_term_match.group(1)
            if not coeff or coeff == '-':
                coeff = '-1' if coeff == '-' else '1'
            return f"{coeff}\\pi"
        frac_pi_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}\\?pi$', expr)
        if frac_pi_match:
            num, den = frac_pi_match.groups()
            return f"\\frac{{{num}}}{{{den}}}\\pi"
        frac_match = re.match(r'^\\frac{([^{}]+)}{([^{}]+)}$', expr)
        if frac_match:
            num, den = frac_match.groups()
            return f"\\frac{{{num}}}{{{den}}}"
    except:
        return expr.lower()

def normalize_interval_bound(bound: str) -> str:
    if '\\infty' in bound:
        sign = '-' if bound.startswith('-') else ''
        return f"{sign}\\infty"
    return normalize_answer(bound) or bound

def normalize_interval(interval_str: str) -> str:
    try:
        interval_str = ''.join(interval_str.split())
        match = re.match(r'^\\left?([\[\(])(.*?),(.*?)\\right?([\]\)])$', interval_str)
        if not match:
            match = re.match(r'^([\[\(])(.*?),(.*?)([\]\)])$', interval_str)
            if not match:
                return interval_str
        left_bracket, left_bound, right_bound, right_bracket = match.groups()
        norm_left = normalize_interval_bound(left_bound)
        norm_right = normalize_interval_bound(right_bound)
        return f"\\left{left_bracket}{norm_left},{norm_right}\\right{right_bracket}"
    except:
        return interval_str

def normalize_ordered_tuple(tuple_str: str) -> str:
    try:
        tuple_str = tuple_str.replace('\\dfrac', '\\frac')
        tuple_str = tuple_str.replace('\\left', '').replace('\\right', '')
        tuple_str = re.sub(r'\\?\s+', '', tuple_str)
        inner = tuple_str.strip('()')
        parts = inner.split(',')
        normalized_parts = [normalize_answer(part.strip()) for part in parts if normalize_answer(part.strip())]
        return f"({','.join(normalized_parts)})"
    except:
        return None

def normalize_answer(answer: str) -> str:
    if answer is None:
        return ""
    answer = re.sub(r'\\text{[^}]+(?:inches|feet|meters|cm|m|kg|ft|in|lb|oz|ml|L|per|second|minute|hour)[^}]*}', '', answer)
    answer = re.sub(r'(?<!\\)\s+', '', answer)
    ordered_pair_match = re.match(r'^(?:\\left)?\((.*?)(?:\\right)?\)$', answer)
    if ordered_pair_match:
        content = ordered_pair_match.group(1)
        parts = content.split(',')
        normalized_parts = [normalize_answer(part) for part in parts if normalize_answer(part)]
        return f"({','.join(normalized_parts)})"
    answer = ''.join(answer.split())
    if not answer:
        return None
    pm_match = re.match(r'^(.*?)(?:\\pm|-)(.*?)$', answer)
    if pm_match:
        left, right = pm_match.groups()
        norm_left = normalize_answer(left) if left else ""
        norm_right = normalize_answer(right) if right else ""
        if norm_left or norm_right:
            return f"{norm_left}\\pm{norm_right}"
    trig_match = re.match(r'^\\(?:sin|cos|tan|cot|sec|csc)\s*([a-zA-Z])$', answer)
    if trig_match:
        variable = trig_match.group(1)
        func_name = re.match(r'^\\(.*?)(?:\s|$)', answer).group(1)
        return f"\\{func_name}{variable}"
    text_match = re.match(r'^(?:\\text{)?([A-Za-z]+)(?:})?$', answer)
    if text_match:
        return text_match.group(1).lower()
    if (answer.startswith('\\left[') or answer.startswith('\\left(') or
        answer.startswith('[') or answer.startswith('(')) and \
       (answer.endswith('\\right]') or answer.endswith('\\right)') or
        answer.endswith(']') or answer.endswith(')')):
        return normalize_interval(answer)
    if answer.startswith('\\begin{pmatrix}') and answer.endswith('\\end{pmatrix}'):
        return normalize_matrix(answer)
    answer = answer.replace('\\dfrac', '\\frac')
    if '\\frac' in answer or '/' in answer:
        return normalize_fraction(answer)
    neg_sqrt_match = re.match(r'^-\\sqrt\{?(\d+)\}?$', answer)
    if neg_sqrt_match:
        num = neg_sqrt_match.group(1)
        return f"-\\sqrt{{{num}}}"
    sqrt_match = re.match(r'^(\d*)?\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_match:
        coeff, num = sqrt_match.groups()
        coeff = coeff if coeff else '1'
        return f"\\sqrt{{{num}}}" if coeff == '1' else f"{coeff}\\sqrt{{{num}}}"
    sqrt_with_coeff_match = re.match(r'^(\d+)\\sqrt\{?(\d+)\}?$', answer)
    if sqrt_with_coeff_match:
        coeff, num = sqrt_with_coeff_match.groups()
        return f"{coeff}\\sqrt{{{num}}}"
    base_match = re.match(r'^(\d+)(?:_\{?(\d+)\}?|_(\d+))$', answer)
    if base_match:
        number, base1, base2 = base_match.groups()
        base = base1 if base1 else base2
        return f"{number}_{base}"
    percent_match = re.match(r'^(\d+(?:\.\d*)?)\s*\\?%$', answer)
    if percent_match:
        return normalize_number(percent_match.group(1))
    unit_match = re.match(r'^(\d+(?:\.\d*)?)\s*(?:(?:\\[,\s])|,)?\s*(?:\\\\)?(?:\\text{(\w+)}|\\?(?:cm|m|kg|ft|in|lb|oz|ml|L))$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    currency_match = re.match(r'^\\?\$?([\d,]+\.?\d*)$', answer)
    if currency_match:
        return normalize_number(currency_match.group(1))
    if re.match(r'^-?[\d,]+$', answer):
        return normalize_number(answer)
    unit_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:\\(?:mbox|text|hbox|displaystyle)\{[^}]+\})?(?:\^?\d)?$', answer)
    if unit_match:
        return normalize_number(unit_match.group(1))
    mc_match = re.match(r'^\\text{\(?([A-Za-z])\)?}$|^\(?([A-Za-z])\)?$', answer)
    if mc_match:
        return (mc_match.group(1) or mc_match.group(2)).lower()
    degree_match = re.match(r'^(-?[\d,]+(?:\.\d*)?)\s*(?:(?:\^?\\circ)|(?:{\\circ})|(?:°))?$', answer)
    if degree_match:
        return normalize_number(degree_match.group(1))
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    try:
        return normalize_algebraic_expression(answer)
    except:
        pass
    answer = answer.replace('\\left', '').replace('\\right', '')
    answer = answer.replace('\\(', '(').replace('\\)', ')')
    answer = answer.replace('\\[', '[').replace('\\]', ']')
    answer = answer.replace('\\{', '{').replace('\\}', '}')
    answer = re.sub(r'\\sqrt\{?(\d+)\}?', r'\\sqrt{\1}', answer)
    answer = re.sub(r'\\sqrt{([^{}]+)}', r'\\sqrt\1', answer)
    if re.match(r'^\d+\\%$', answer) or re.match(r'^\d+$', answer):
        answer = re.sub(r'\\%$', '', answer)
    answer = re.sub(r'\\text{([^{}]+)}', r'\1', answer)
    while len(answer) >= 2 and answer[0] == '{' and answer[-1] == '}':
        if '\\frac' in answer:
            break
        answer = answer[1:-1]
    return answer.lower() if answer else None

def compare_answers(correct_answer: str, predicted_answer: Optional[str]) -> bool:
    if predicted_answer is None:
        return False
    if numerically_equal(correct_answer, predicted_answer):
        return True
    normalized_correct = normalize_answer(correct_answer)
    normalized_predicted = normalize_answer(predicted_answer)
    if not normalized_correct or not normalized_predicted:
        return False
    if normalized_correct == "" and normalized_predicted == "":
        return False
    if ('\\left[' in normalized_correct or '\\left(' in normalized_correct) and \
       ('\\left[' in normalized_predicted or '\\left(' in normalized_predicted):
        return normalized_correct == normalized_predicted
    return normalized_correct == normalized_predicted

# Load existing results
def load_existing_results(filename: str) -> list[Dict]:
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# Save a single result
def save_result(filename: str, result: Dict):
    results = load_existing_results(filename)
    results.append(result)
    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)

# Analyze and print results
def analyze_results(results: list[Dict]):
    total = len(results)
    correct = sum(1 for r in results if r['is_correct'])
    accuracy = correct / total if total > 0 else 0
    print("\n=== Results Summary ===")
    print(f"Total problems: {total}")
    print(f"Correct answers: {correct}")
    print(f"Accuracy: {accuracy:.2%}")
    print("\n=== Incorrect Problems ===")
    for r in results:
        if not r['is_correct']:
            print(f"Problem {r['index']}:")
            print(f"Expected: {r['correct_answer']}")
            print(f"Predicted: {r['predicted_answer']}")
            print("---")

# Main evaluation function
def evaluate():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    t=0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        t += 1
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])  # Extract from 'solution', not 'answer'
        response = get_llm_response(problem_text)
        predicted_answer = extract_answer(response)
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"cnt :  {cnt} idx: {t}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)



# Customizable CoT Prompt Template
* modify cot prompt then evaluate on math benchmark


In [ ]:
# final answer should be in this format: (because of extract_answer function you can change it if you want)
#\\[
#\\boxed{your_answer_here}
#\\]

COT_PROMPT = """Solve the following math problem step by step, clearly explaining your reasoning.
At the end, write only the final answer enclosed in LaTeX format as shown:

\\[
\\boxed{your_final_answer}
\\]

Make sure the entire answer is inside the \\boxed{}.
"""

* generate response with cot prompt

In [ ]:
import requests

def get_COT_response(problem):
    prompt = COT_PROMPT + "\n" + problem
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": 1900,
    "temperature": 0.3
    }
    response = requests.post(url, json=payload)
    return response.json()['choices'][0]['message']['content'].strip()

# Evaluate CoT
* modify response generation part to evalute this method.

In [ ]:
def evaluate_cot():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_cot.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])

        # TODO: Generate a response with cot
        response = get_COT_response(problem_text)
        predicted_answer = extract_answer(response)
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_cot()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating problems:   0%|          | 1/500 [00:10<1:28:58, 10.70s/it]

...

corrects :  12 idx: 27


Evaluating problems:   6%|▌         | 29/500 [11:10<2:37:15, 20.03s/it]

corrects :  12 idx: 28


Evaluating problems:   6%|▌         | 30/500 [11:26<2:59:08, 22.87s/it]

corrects :  13 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 13
Accuracy: 43.33%

=== Incorrect Problems ===
Problem 1:
Expected: p - q
Predicted: None
---
Problem 4:
Expected: \text{Evelyn}
Predicted: None
---
Problem 9:
Expected: 4
Predicted: None
---
Problem 10:
Expected: 2220
Predicted: None
---
Problem 11:
Expected: \frac{3}{56}
Predicted: None
---
Problem 12:
Expected: 284
Predicted: None
---
Problem 14:
Expected: \sqrt{51}
Predicted: 5
---
Problem 15:
Expected: 6 - 5i
Predicted: None
---
Problem 17:
Expected: \pi
Predicted: None
---
Problem 18:
Expected: 28
Predicted: None
---
Problem 19:
Expected: 3
Predicted: None
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: None
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 25:
Expected: 1,-2
Predicted: None
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Best-of-N

The Best-of-N approach generates several candidate responses for a problem and then selects the one with the highest average token log-likelihood. This ensures that the final answer, formatted within the `\boxed{}` command, is not only correct in presentation but also statistically the most reliable.


In [10]:
SYSTEM_PROMPT = '''You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\\[
\\boxed{your_answer_here}
\\]

The entire answer should be contained completely within the \\boxed{} command.'''

def get_response_logprobs(prompt, max_tokens=1900, temperature=0.3):
    url = "http://localhost:8000/v1/chat/completions"

    payload = {
        "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }

        ],
    "max_tokens": max_tokens,
    "temperature": temperature,
    "logprobs": True,
    }
    response = requests.post(url, json=payload)
    return response.json()

def best_of_n_response(problem, N=5):
    best_answer = None
    best_avg_likelihood = float('-inf')
    best_responses = []
    prompt = SYSTEM_PROMPT + "\n" + problem

    for t in range(N):
        # TODO: Generate a response
        response = get_response_logprobs(prompt)
        # TODO:  Iterate over each choice in the response and append lobprob of each tocken to token_logprobs (you can see a sample of response to see how to extract the token logprobs)
        for choice in response['choices']:
            content = choice['message']['content'].strip()
            token_logprobs = []
            for cnt in choice['logprobs']['content']:
                if 'logprob' in cnt:
                    token_logprobs.append(cnt['logprob'])
            if len(token_logprobs) == 0:
                continue
            # TODO: Calculate the average log-likelihood and store the response, answer(that is extracted with extract_answer()), and average log-likelihood
            avg_likelihood = sum(token_logprobs) / len(token_logprobs)
            answer = extract_answer(content)
            if answer == None:
                continue
            best_responses.append((answer, avg_likelihood))


    # TODO: Group the responses by the answer (multiple responses can have the same answer)
    grouped_responses = {}
    for (answer, avg_likelihood) in best_responses:
        if answer not in grouped_responses:
            grouped_responses[answer] = []
        grouped_responses[answer].append(avg_likelihood)

    for answer in grouped_responses:
        likelihoods = grouped_responses[answer]
        grouped_responses[answer] = sum(likelihoods) / len(likelihoods)
    # TODO: Find the best answer based on the average likelihood
    for (answer, avg_likelihood) in grouped_responses.items():
        if avg_likelihood > best_avg_likelihood:
            best_avg_likelihood = avg_likelihood
            best_answer = answer

    return best_answer

# Evaluate best of n

* modify response generation part to evalute this method.

In [ ]:
def evaluate_best_of_n():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_best_of_n.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes:
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        # TODO: ##########################################################
        response = best_of_n_response(problem_text)
        predicted_answer = response
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_best_of_n()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating problems:   0%|          | 1/500 [00:57<7:54:56, 57.11s/it]

...

corrects :  13 idx: 27


Evaluating problems:   6%|▌         | 29/500 [55:08<15:30:38, 118.55s/it]

corrects :  13 idx: 28


Evaluating problems:   6%|▌         | 30/500 [55:51<14:35:10, 111.72s/it]

corrects :  14 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 14
Accuracy: 46.67%

=== Incorrect Problems ===
Problem 1:
Expected: p - q
Predicted: None
---
Problem 4:
Expected: \text{Evelyn}
Predicted: 1
---
Problem 5:
Expected: 42
Predicted: 42\ \text{inches}
---
Problem 9:
Expected: 4
Predicted: None
---
Problem 10:
Expected: 2220
Predicted: 222
---
Problem 11:
Expected: \frac{3}{56}
Predicted: None
---
Problem 12:
Expected: 284
Predicted: 286
---
Problem 17:
Expected: \pi
Predicted: \frac{\pi}{3}
---
Problem 18:
Expected: 28
Predicted: None
---
Problem 19:
Expected: 3
Predicted: None
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: None
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 25:
Expected: 1,-2
Predicted: None
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Beam Search

This cell implements a beam search strategy for generating candidate reasoning chains. The method generates multiple continuations at each reasoning step, scoring each candidate based on its average token log-likelihood. By retaining and expanding only the top candidates, the approach efficiently searches for the most promising chain-of-thought that leads to the final answer in the required format.


In [ ]:
def call_qwen_model_raw(prompt,step_num, temperature=0.6):
    """
    Sends a request to the local Qwen endpoint and returns the generated text
    along with the average token log-probability.
    """
    # Build the prompt. We assume the sample already contains the SYSTEM_PROMPT. you can modify max_tokens for different steps
    max_tokens = [500, 1000, 1500, 2000]

    # TODO: Send a request to the Qwen model and get the response
    response = get_response_logprobs(prompt, max_tokens[step_num], temperature=temperature)

    # TODO:  Iterate over each choice in the response and append lobprob of each tocken to token_logprobs
    token_logprobs = []

    choice = response['choices'][0]
    output_text = choice['message']['content'].strip()
    token_logprobs = []
    for cnt in choice['logprobs']['content']:
        if 'logprob' in cnt:
            token_logprobs.append(cnt['logprob'])

    # TODO: Calculate the average log-likelihood

    avg_token_prob = sum(token_logprobs) / len(token_logprobs)

    return output_text, avg_token_prob, len(token_logprobs)



class BeamCandidate:
    def __init__(self, sequence, cumulative_log_prob, step_scores, finished=False,num_token = 0):
        self.sequence = sequence
        self.cumulative_log_prob = cumulative_log_prob
        self.step_scores = step_scores
        self.finished = finished
        self.num_token = num_token

    def __repr__(self):
        return (f"BeamCandidate(score={self.cumulative_log_prob:.3f}, finished={self.finished}, "
                f"sequence={self.sequence})")



def generate_reasoning_steps(context, step_num, top_k):
    """
    For a given candidate reasoning chain (context), generate top_k candidate continuations
    for the current reasoning step (from 1 to 5). Each candidate is verified using the average
    token logprob as a proxy for quality.
    """

    # TODO: each step should have a different prompt and the prompt should be added to the context so make a prompt for each step that explains what the step is about
    candidates = []
    for i in range(top_k):
        if step_num == 1:
            candidate_prompt = context + "\n" + "For the 1st step, you should understand what the problem wants, and specify its objectives."
        if step_num == 2:
            candidate_prompt = context + "\n" + "For the 2nd step, you should think and plan how you want to solve the problem (by reasoning)."
        if step_num == 3:
            candidate_prompt = context + "\n" + "For the 3rd and last step, you should fully solve the problem."

        # TODO: call the qwen model to get the output and avg_token_prob
        candidate_step, avg_token_prob, num_token = call_qwen_model_raw(candidate_prompt, step_num)
        finished = True if "\\boxed" in candidate_step else False
        candidates.append((candidate_step, avg_token_prob, num_token, finished))

    return candidates

def beam_search(init_problem_prompt, beam_width=3, max_steps=3, top_k=2):
    """
    Implements a beam search over reasoning steps.
    """
    prompt = init_problem_prompt
    initial_candidate = BeamCandidate(sequence=init_problem_prompt, cumulative_log_prob=0, step_scores=[])
    beams = [initial_candidate]

    for step_num in range(1, max_steps+1):
        new_beams = []

        for candidate in beams:
            if candidate.finished:
                # TODO: Propagate finished candidates unchanged.
                new_beams.append(candidate)
                continue
            step_candidates = generate_reasoning_steps(candidate.sequence, step_num, top_k)
            for (step_text, score, num_token, finished) in step_candidates:
                # TODO: Create a new candidate by appending the step text to the current sequence and updating the log-probability by averaging all token_logprobs after the new step.
                new_candidate = candidate.sequence + "\n" + step_text
                new_logprob = (candidate.cumulative_log_prob * candidate.num_token + score * num_token) / (candidate.num_token + num_token)
                new_scores = candidate.step_scores + [score]
                new_beams.append(BeamCandidate(new_candidate, new_logprob, new_scores, finished, candidate.num_token + num_token))

        if not new_beams:
            break
        # TODO: sort the new beams based on the cumulative_log_prob and put the top beam_width beams in the beams list
        beams = sorted(new_beams, key=lambda x: x.cumulative_log_prob, reverse=True)
        beams = beams[:beam_width]

        if all(beam.finished for beam in beams):
            break
    # TODO: Get the best candidate from the beams list that is finished
    finished_beams = [b for b in beams if b.finished]

    best_candidate = max(finished_beams, key=lambda x: x.cumulative_log_prob) if finished_beams else beams[0]
    return best_candidate

def run_qwen_beam_search(problem, beam_width, max_steps, top_k, log_level):
    """
    sets up the sample prompt, performs beam search,
    and extracts the final answer.
    """
    # TODO: Set the initial prompt to the problem and run the beam search to get the best candidate

    initial_prompt = SYSTEM_PROMPT + "\n" + problem
    best_candidate = beam_search(initial_prompt, beam_width, max_steps, top_k)
    final_answer = extract_answer(best_candidate.sequence) if best_candidate.finished else None

    return final_answer


# Evaluate beam search
* modify response generation part to evalute this method.

In [ ]:
def evaluate_beam_search():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_beam_search.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes :
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        ##########################################################
        # TODO: Generate a response with beam search
        response = run_qwen_beam_search(problem_text, 3, 3, 2, "")
        predicted_answer = response
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [ ]:
evaluate_beam_search()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Evaluating problems:   0%|          | 1/500 [00:31<4:23:05, 31.64s/it]

...

corrects :  18 idx: 27


Evaluating problems:   6%|▌         | 29/500 [1:02:56<17:03:04, 130.33s/it]

corrects :  18 idx: 28


Evaluating problems:   6%|▌         | 30/500 [1:03:15<16:31:08, 126.53s/it]

corrects :  19 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 19
Accuracy: 63.33%

=== Incorrect Problems ===
Problem 9:
Expected: 4
Predicted: None
---
Problem 11:
Expected: \frac{3}{56}
Predicted: \dfrac{1}{9}
---
Problem 12:
Expected: 284
Predicted: 260
---
Problem 17:
Expected: \pi
Predicted: 
---
Problem 18:
Expected: 28
Predicted: 
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 24:
Expected: 10
Predicted: 49
---
Problem 25:
Expected: 1,-2
Predicted: 1
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


## Self-Refinement

This approach begins by generating an initial solution using the given prompt. It then iteratively refines this output by providing the model with targeted feedback and asking it to improve its response. The process continues until the feedback indicates that no further refinement is necessary, ensuring that the final answer—properly formatted within the `\boxed{}` command—is as accurate and well-reasoned as possible.


In [22]:
SYSTEM_PROMPT = '''You are solving mathematics problems.

Please think step by step.

Important: Always end your solution with the final answer in this format:

\\[
\\boxed{your_answer_here}
\\]

The entire answer should be contained completely within the \\boxed{} command.'''

def generate_content(prompt):
    # TODO: Send a request to the Qwen model and get the response

    response = get_response_logprobs(prompt, max_tokens=2200)
    output_text = response['choices'][0]['message']['content'].strip()

    return output_text

def self_refine(problem, max_iter=2):

    prompt = SYSTEM_PROMPT + "\n" + problem

    # TODO: Generate the initial output using generate_content with the full prompt.
    current_output = generate_content(prompt)

    for iteration in range(max_iter):
        # TODO: Provide a feedback prompt that asks the model to analyze current_output.
        #       - Include both the original prompt and the current output
        #       - speicfy the format if the feedback is needed so you can parse it later
        feedback_prompt = f'''
        You are an expert math reviewer.

        You will be shown a math problem and a solution. Your job is to analyze the solution, find any mistakes, and decide if the answer should be refined.

        Respond in the following format:

        Needs refinement: <yes/no>

        Justification:
        <your explanation here>

        ---

        Problem:
        {prompt}

        Proposed Solution:
        {current_output}
        '''
        feedback_output = generate_content(feedback_prompt)

        # TODO: Parse the feedback response to determine if refinement is needed.
        parsed = re.search(r"Needs refinement:\s*(yes|no)", feedback_output, re.IGNORECASE)
        if parsed is None:
            break

        refinement_needed = parsed and parsed.group(1).strip().lower() == "yes"

        if not refinement_needed:
            break

        # TODO: If refinement is needed:
        #       - Create a refine prompt that includes the original prompt, current output, and the feedback.
        #       - Send this refine prompt to the model using generate_content to obtain a refined output.
        #       - Update current_output with the refined output.
        refinement_prompt = f'''
        You previously answered the following math problem:
        Problem:
        {prompt}

        Your previous solution was:
        {current_output}

        A reviewer provided the following feedback:
        {feedback_output}

        Please revise your solution accordingly. Be sure to end with the final answer in this format:

        \\[
        \\boxed{{your_answer_here}}
        \\]
        '''
        refined_output = generate_content(refinement_prompt)
        if refined_output.strip() == current_output.strip():
            break
        current_output = refined_output

    # TODO: Extract the final answer from current_output (e.g., using an extract_answer function).
    answer = extract_answer(current_output)
    return answer



# Evaluate Self-Refinement
* modify response generation part to evalute this method.

In [23]:
def evaluate_self_refiner():
    os.makedirs("results", exist_ok=True)
    results_file = "evaluation_results_math500_deepseek_self_refiner.json"
    dataset = load_math500_dataset()
    existing_results = load_existing_results(results_file)
    processed_indexes = {result['index'] for result in existing_results}
    cnt = 0
    for idx, item in enumerate(tqdm(dataset, desc="Evaluating problems")):
        if idx in processed_indexes :
            continue
        if idx >= 30:
          break
        problem_text = item['problem']
        correct_answer = extract_answer(item['solution'])
        ##########################################################
        # TODO: Generate a response with self_refine
        response = self_refine(problem_text, 4)
        predicted_answer = response
        ##########################################################
        is_correct = compare_answers(correct_answer, predicted_answer)
        result = {
            "index": idx,
            "problem": problem_text,
            "response": response,
            "correct_answer": correct_answer,
            "predicted_answer": predicted_answer,
            "is_correct": is_correct
        }
        save_result(results_file, result)
        if is_correct:
          cnt += 1
        print(f"corrects :  {cnt} idx: {idx}")
    final_results = load_existing_results(results_file)
    analyze_results(final_results)

In [24]:
evaluate_self_refiner()

Evaluating problems:   0%|          | 2/500 [01:17<5:21:23, 38.72s/it]

corrects :  0 idx: 1


Evaluating problems:   1%|          | 3/500 [01:40<4:25:35, 32.06s/it]

corrects :  1 idx: 2


Evaluating problems:   1%|          | 4/500 [01:57<3:40:59, 26.73s/it]

...

corrects :  12 idx: 27


Evaluating problems:   6%|▌         | 29/500 [20:17<5:13:29, 39.94s/it]

corrects :  12 idx: 28


Evaluating problems:   6%|▌         | 30/500 [20:36<5:22:47, 41.21s/it]

corrects :  13 idx: 29

=== Results Summary ===
Total problems: 30
Correct answers: 14
Accuracy: 46.67%

=== Incorrect Problems ===
Problem 1:
Expected: p - q
Predicted: None
---
Problem 4:
Expected: \text{Evelyn}
Predicted: 3.6 \text{ km/h}
---
Problem 9:
Expected: 4
Predicted: None
---
Problem 11:
Expected: \frac{3}{56}
Predicted: None
---
Problem 12:
Expected: 284
Predicted: 286
---
Problem 15:
Expected: 6 - 5i
Predicted: None
---
Problem 17:
Expected: \pi
Predicted: 
---
Problem 18:
Expected: 28
Predicted: 
---
Problem 19:
Expected: 3
Predicted: None
---
Problem 20:
Expected: 6+9i
Predicted: 6 + 9i
---
Problem 21:
Expected: 13535
Predicted: None
---
Problem 22:
Expected: 5
Predicted: None
---
Problem 23:
Expected: x=5
Predicted: 5
---
Problem 25:
Expected: 1,-2
Predicted: None
---
Problem 26:
Expected: 144
Predicted: None
---
Problem 28:
Expected: -2 + 7i
Predicted: -2 + 7i
---


<font color='red'>

## MAKE SURE TO READ THE FIRST MARKDOWN CELL! :D